In [ ]:
%pip install -U scikit-learn

In [1]:
# importando bibliotecas necessárias
import pandas as pd
import numpy as np

# graphs
import matplotlib.pyplot as plt
import seaborn as sns

# criação de pipeline para tratamento do conjunto de dados
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

#modelo
from sklearn.linear_model import LinearRegression

In [126]:
# agora uma função que vai printar bonitinho as métricas
def print_metrics(model_name: str, y_test: np.ndarray, y_pred: np.ndarray) -> str:
  """
  Returns the accuracy, precision and recall scores of the model
      
      Arguments:
          model_name {str} -- name of the model
          y_test {array} -- array of the test labels
          y_pred {array} -- array of the predicted labels
          y_test_pca {array} -- array of the test labels after PCA
          y_pred_pca {array} -- array of the predicted labels after PCA
      
      Returns:
          A formatted string containing the accuracy, precision and recall scores of the model for both non-PCA and PCA.
  
  """
  #imports
  from sklearn.metrics import mean_absolute_error, mean_squared_error

  return (f'''

Mean Absolute Error (MAE) do Modelo {model_name} sem PCA: {mean_absolute_error(y_test, y_pred):.4f}

Mean Squared Error (MSE) do Modelo {model_name} sem PCA: {mean_squared_error(y_test, y_pred, squared=True):.4f}

Root Mean Squared Error (RMSE) do Modelo {model_name} sem PCA: {mean_squared_error(y_test, y_pred, squared=False):.4f}
''')

In [131]:
from warnings import simplefilter
#from sklearn.exceptions import UserWarning

simplefilter("ignore", category=UserWarning)

## 1. Análise exploratória de dados (1.0 ponto)

Realize a análise exploratória de dados. Observe cada variável, como se comportam em relação a outras variáveis independentes e com a variável dependente.


In [27]:
# carregando dataset de teste

dataframe = pd.read_csv('dado/train.csv')
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [25]:
dataframe.fillna(0, inplace=True)

In [28]:
dataframe.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [6]:
dataframe['Alley'].fillna('None', inplace=True)
dataframe['Fence'].fillna('None', inplace=True)
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

info on onehotencoder:

It is also possible to encode each column into n_categories - 1 columns instead of n_categories columns by using the drop parameter. This parameter allows the user to specify a category for each feature to be dropped. This is useful to avoid co-linearity in the input matrix in some classifiers. Such functionality is useful, for example, when using non-regularized regression (LinearRegression), since co-linearity would cause the covariance matrix to be non-invertible:

In [21]:
#filter categorical data
column_cat = dataframe.select_dtypes(include=[object])
#column_cat.info()

In [25]:
dataframe[col].columns == column_cat.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [26]:

#encoding categorical data
encoder = OneHotEncoder(dtype=int, handle_unknown='ignore')
encoder.fit(column_cat)


OneHotEncoder(dtype=<class 'int'>, handle_unknown='ignore')

In [29]:
encoder.n_features_in_

43

In [31]:
data_encoded = encoder.fit_transform(column_cat)
len(data_encoded.shape)

2

In [35]:
len(data_encoded.toarray())

1460

In [36]:
data_encoded

<1460x268 sparse matrix of type '<class 'numpy.int64'>'
	with 62780 stored elements in Compressed Sparse Row format>

In [28]:

col = column_cat.columns
dataframe[col] = encoder.fit_transform(column_cat)

dataframe.info()

# encoder = OrdinalEncoder()
# categoricos = dataframe_cp.select_dtypes(include=[object])
# data_encoded = encoder.fit_transform(categoricos)
# colunas = categoricos.columns
# dataframe_cp[colunas] = encoder.fit_transform(categoricos)
# dataframe_cp.head()

ValueError: Columns must be same length as key

## 2. Levantamento de hipóteses (1.0 ponto)

Descreva quais hipóteses você observou ao fazer a análise exploratória de dados.



## 3. Apresentação das ideias obtidas (1.0 ponto)

Apresente com gráficos as suas observações e a descreva cada gráfico.


## 4. Preparação dos dados (1.0 ponto)

Faça um processamento nos dados, preenchendo valores faltantes, removendo dados ou variáveis inconsistentes e normalizado os dados



#### Steps
- Outliers
- Encoding
- Impute Missing Values
- Scaler
- PCA
- Model
- Cross-Val

In [35]:
# criação de pipeline para tratamento do conjunto de dados

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, LassoCV

from sklearn.compose import make_column_selector

from sklearn.preprocessing import StandardScaler



In [74]:
col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

dataframe.columns[dataframe.isnull().any()].tolist()

'MiscFeature' possui apenas 51 observações e 'MiscVal' corresponde ao valor das features observadas em 'MiscFeature', por tanto, ambas serão removidas.

In [123]:
# construindo pipeline para o modelo de regressão usando LassoCV

numeric_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler())
        ]
)

categorical_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False))
        ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer, col_num),
    ("cat", categorical_transformer, col_tex)
  ]
)

model = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", LassoCV(cv=5))
  ]
)

model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f86108f7190>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f86108f7fa0>)])),
                ('lassocv', LassoCV(cv=5))])

In [37]:
from sklearn.model_selection import train_test_split

X = dataframe.drop(['SalePrice'], axis=1)
y = dataframe.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [124]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

/home/joao/GitHub/Blue/trilha_dados/dados_mod1/.projeto2/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [14, 29] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [127]:
print(print_metrics(model_name='Pipeline Linear Regression', y_test=y_test, y_pred=y_pred))



Mean Absolute Error (MAE) do Modelo Pipeline Linear Regression sem PCA: 17523.0653

Mean Squared Error (MSE) do Modelo Pipeline Linear Regression sem PCA: 866285046.5730

Root Mean Squared Error (RMSE) do Modelo Pipeline Linear Regression sem PCA: 29432.7207



In [132]:
print("model score R-squared: %.3f" % model.score(X_test, y_test))

model score R-squared: 0.887


In [164]:
preprocessor_fitted = model['preprocessor']
#preprocessor_fitted.named_transformers_

feature_names = (
  list(preprocessor_fitted.named_transformers_['num']['imputer'].get_feature_names_out()) +
  list(preprocessor_fitted.named_transformers_['cat']['one_hot_encoder'].get_feature_names_out())
)

feature_names = np.array(feature_names)

kept = ~np.isclose(model['lassocv'].coef_, 0)
feature_names[kept]


array(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'x0_RM', 'x3_IR2',
       'x4_Bnk', 'x4_HLS', 'x6_CulDSac', 'x7_Mod', 'x8_BrkSide',
       'x8_Crawfor', 'x8_Edwards', 'x8_Gilbert', 'x8_Mitchel', 'x8_NAmes',
       'x8_NWAmes', 'x8_NoRidge', 'x8_NridgHt', 'x8_Somerst',
       'x8_StoneBr', 'x9_Norm', 'x10_PosN', 'x11_1Fam', 'x11_2fmCon',
       'x12_1Story', 'x12_2Story', 'x13_Gable', 'x14_ClyTile',
       'x15_BrkFace', 'x15_HdBoard', 'x16_ImStucc', 'x16_Plywood',
       'x17_BrkFace', 'x17_None', 'x18_Ex', 'x18_TA', 'x20_BrkTil',
 

In [165]:
len(feature_names[kept])


90

## 5. Criação e seleção de características (1.0 ponto)

Observando as variáveis disponíveis, é possível criar novas variáveis? Ou trazer de um dado externo novos dados para agregar aos existentes? É possível selecionar uma variável ou grupo de variáveis que melhor explica a variável dependente?


## 6. Decisão do modelo (1.0 ponto)

Dada toda a análise realizada nos pontos anteriores, qual modelo melhor se adequa a este problema? E por quê?



## 7. Modelagem (1.0 ponto)

Divida os dados em teste e treino, construa o seu modelo e teste alterar os parâmetros utilizados por ele.



## 8. Histórico (1.0 pontos)

Preencha a tabela abaixo com cada teste realizado com seu modelo. Não se preocupe caso voce mude de ideia sobre qual modelo testar, mas não deixe de testá-lo exaustivamente e, caso precise, retorne para o passo de criação e seleção de características.


## 9. Resultado (2.0 pontos)

Reúna os seus achados, os valores das métricas do seu melhor modelo obtido e o melhor conjunto de variáveis. Escreva abaixo um relatório que seria enviado para uma diretoria de uma empresa sobre seu trabalho.
